In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from collections import defaultdict
from sklearn.feature_selection import (
    VarianceThreshold, chi2, f_classif, mutual_info_classif, 
    SelectKBest, RFE, SelectFromModel, SequentialFeatureSelector
)
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from boruta import BorutaPy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import (
    Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
)
from keras.callbacks import EarlyStopping

warnings.filterwarnings('ignore')

# Load dataset
def load_data(file_path):
    print("Loading dataset...")
    df = pd.read_csv(file_path)
    X = df.drop(columns=['COVID-19'])
    y = df['COVID-19']
    print(f"Dataset loaded: {X.shape[0]} samples, {X.shape[1]} features")
    print(f"Class distribution: {dict(y.value_counts())}")
    return X, y

# Define all feature selection techniques
def get_feature_selectors(X, y, n_features=10):
    print("Initializing feature selection techniques...")
    feature_selectors = {
        "1. Chi-Square": SelectKBest(chi2, k=n_features),
        "2. Mutual Information": SelectKBest(mutual_info_classif, k=n_features),
        "3. Recursive Feature Elimination": RFE(
            estimator=LogisticRegression(solver='liblinear', max_iter=1000, random_state=42),
            n_features_to_select=n_features
        ),
        "4. Lasso": SelectFromModel(
            Lasso(alpha=0.01, random_state=42), max_features=n_features
        ),
        "5. Random Forest Importance": SelectFromModel(
            RandomForestClassifier(n_estimators=100, random_state=42), max_features=n_features
        ),
        "6. Boruta": BorutaPy(
            RandomForestClassifier(n_estimators=100, random_state=42),
            n_estimators='auto', verbose=0, random_state=42
        ),
        "7. Correlation-based": None,  # Custom implementation
        "8. Sequential Forward Selection": SequentialFeatureSelector(
            RandomForestClassifier(n_estimators=50, random_state=42),
            n_features_to_select=n_features,
            direction='forward'
        ),
        "9. XGBoost Importance": SelectFromModel(
            XGBClassifier(n_estimators=100, random_state=42), max_features=n_features
        ),
        "10. LightGBM Importance": SelectFromModel(
            LGBMClassifier(n_estimators=100, random_state=42), max_features=n_features
        )
    }
    return feature_selectors

# Function to select top features using each technique
def select_features(X, y, technique_name, selector, n_features=10):
    print(f"Selecting features using {technique_name}...")
    feature_names = X.columns.tolist()
    
    # Handle special case for Correlation-based selection
    if technique_name == "7. Correlation-based":
        correlations = []
        for col in X.columns:
            corr = np.abs(pd.crosstab(X[col], y, normalize='columns').iloc[1, 1] - 
                          pd.crosstab(X[col], y, normalize='columns').iloc[1, 0])
            correlations.append((col, corr))
        
        correlations.sort(key=lambda x: x[1], reverse=True)
        selected_features = [item[0] for item in correlations[:n_features]]
        feature_importances = [item[1] for item in correlations[:n_features]]
        
    elif technique_name == "6. Boruta":
        X_array = X.values
        selector.fit(X_array, y)
        selected_mask = selector.support_
        ranking = selector.ranking_
        
        feature_ranking = [(feature, rank) for feature, rank, mask in 
                          zip(feature_names, ranking, selected_mask) if mask]
        feature_ranking.sort(key=lambda x: x[1])
        
        if len(feature_ranking) < n_features:
            additional = [(f, r) for f, r, m in 
                         zip(feature_names, ranking, selected_mask) if not m]
            additional.sort(key=lambda x: x[1])
            feature_ranking.extend(additional[:n_features-len(feature_ranking)])
        
        feature_ranking = feature_ranking[:n_features]
        selected_features = [item[0] for item in feature_ranking]
        feature_importances = [1.0/item[1] for item in feature_ranking]
    
    else:
        try:
            selector.fit(X, y)
            selected_mask = selector.get_support()
            selected_features = [f for f, selected in zip(feature_names, selected_mask) if selected]
            
            if hasattr(selector, 'estimator_') and hasattr(selector.estimator_, 'feature_importances_'):
                feature_importances = selector.estimator_.feature_importances_[selected_mask]
            elif hasattr(selector, 'scores_'):
                feature_importances = selector.scores_[selected_mask]
            else:
                feature_importances = np.ones(len(selected_features))
                
        except Exception as e:
            print(f"Error with {technique_name}: {str(e)}")
            selected_features = feature_names[:n_features]
            feature_importances = np.ones(n_features)
    
    if len(selected_features) > n_features:
        selected_features = selected_features[:n_features]
        feature_importances = feature_importances[:n_features]
    elif len(selected_features) < n_features:
        remaining = [f for f in feature_names if f not in selected_features]
        selected_features.extend(remaining[:n_features-len(selected_features)])
        feature_importances = list(feature_importances) + [0] * (n_features - len(feature_importances))
    
    print(f"Top {len(selected_features)} features selected by {technique_name}:")
    for i, (feature, importance) in enumerate(zip(selected_features, feature_importances)):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    return selected_features, feature_importances

# Build the CNN model for a specific set of features
def build_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2, padding='same'),
        Dropout(0.2),
        
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2, padding='same'),
        Dropout(0.3),
        
        Flatten(),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Train and evaluate model with k-fold cross validation
def train_and_evaluate(X, y, selected_features, technique_name, k=5):
    print(f"\nTraining CNN with features selected by {technique_name}")
    
    X_selected = X[selected_features].values
    X_selected = X_selected.reshape(X_selected.shape[0], X_selected.shape[1], 1)
    y_values = y.values
    
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'auc': []
    }
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=0,
        mode='min',
        restore_best_weights=True
    )
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X_selected)):
        print(f"Training fold {fold+1}/{k}...")
        
        X_train, X_test = X_selected[train_idx], X_selected[test_idx]
        y_train, y_test = y_values[train_idx], y_values[test_idx]
        
        model = build_cnn_model((X_selected.shape[1], 1))
        model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            verbose=0,
            validation_split=0.2,
            callbacks=[early_stop]
        )
        
        y_pred = model.predict(X_test)
        y_pred_classes = (y_pred > 0.5).astype(int)
        
        metrics['accuracy'].append(accuracy_score(y_test, y_pred_classes))
        metrics['precision'].append(precision_score(y_test, y_pred_classes))
        metrics['recall'].append(recall_score(y_test, y_pred_classes))
        metrics['f1'].append(f1_score(y_test, y_pred_classes))
        try:
            metrics['auc'].append(roc_auc_score(y_test, y_pred))
        except:
            metrics['auc'].append(0.5)
    
    avg_metrics = {metric: np.mean(values) for metric, values in metrics.items()}
    std_metrics = {metric: np.std(values) for metric, values in metrics.items()}
    
    print(f"\nResults for {technique_name}:")
    for metric, value in avg_metrics.items():
        print(f"Average {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")
    
    return avg_metrics

# Plot comparison bar chart
def plot_comparison(all_results):
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    
    sorted_techniques = sorted(
        all_results.keys(),
        key=lambda x: all_results[x]['accuracy'],
        reverse=True
    )
    
    plt.figure(figsize=(15, 10))
    bar_width = 0.15
    index = np.arange(len(sorted_techniques))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    for i, metric in enumerate(metrics_to_plot):
        values = [all_results[technique][metric] for technique in sorted_techniques]
        plt.bar(
            index + i * bar_width, 
            values, 
            bar_width, 
            label=metric.capitalize(),
            color=colors[i]
        )
    
    plt.xlabel('Feature Selection Technique', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.title('Comparison of Feature Selection Techniques', fontsize=14)
    plt.xticks(
        index + bar_width * 2, 
        [t.split('. ')[1] if '. ' in t else t for t in sorted_techniques],
        rotation=45,
        ha='right'
    )
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)
    plt.ylim(0, 1.0)
    plt.tight_layout()
    
    plt.savefig('feature_selection_comparison.png', dpi=300, bbox_inches='tight')
    print("Comparison chart saved as 'feature_selection_comparison.png'")
    plt.close()

# Main function to run the whole process
def main():
    file_path = "OneDrive/Desktop/Research_Capstone_Project/preprocessed_covid500_final.csv"  # Update with your path
    X, y = load_data(file_path)
    
    feature_selectors = get_feature_selectors(X, y)
    
    all_results = {}
    all_selected_features = {}
    
    for technique_name, selector in feature_selectors.items():
        print("\n" + "="*50)
        print(f"Processing {technique_name}")
        print("="*50)
        
        selected_features, _ = select_features(X, y, technique_name, selector)
        all_selected_features[technique_name] = selected_features
        
        results = train_and_evaluate(X, y, selected_features, technique_name)
        all_results[technique_name] = results
    
    plot_comparison(all_results)
    
    print("\n" + "="*50)
    print("FINAL SUMMARY")
    print("="*50)
    
    sorted_techniques = sorted(
        all_results.keys(),
        key=lambda x: all_results[x]['accuracy'],
        reverse=True
    )
    
    print("\nTechniques ranked by accuracy:")
    for i, technique in enumerate(sorted_techniques):
        print(f"{i+1}. {technique}: {all_results[technique]['accuracy']:.4f}")
    
    best_technique = sorted_techniques[0]
    print(f"\nBest performing technique: {best_technique}")
    print(f"Top features selected by {best_technique}:")
    for feature in all_selected_features[best_technique]:
        print(f"- {feature}")

if __name__ == "__main__":
    main()

Loading dataset...
Dataset loaded: 37028 samples, 20 features
Class distribution: {1.0: 18514, 0.0: 18514}
Initializing feature selection techniques...

Processing 1. Chi-Square
Selecting features using 1. Chi-Square...
Top 10 features selected by 1. Chi-Square:
1. Breathing Problem: 10858.4695
2. Sore throat: 13901.1472
3. Running Nose: 1792.9213
4. Headache: 1768.0620
5. Heart Disease: 4165.6170
6. Diabetes: 2915.7714
7. Hyper Tension: 3682.1903
8. Gastrointestinal : 2622.2793
9. Attended Large Gathering: 9772.2954
10. Family working in Public Exposed Places: 8644.4843

Training CNN with features selected by 1. Chi-Square
Training fold 1/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Training fold 2/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Training fold 3/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Training fold 4/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Training fold 5/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step

Results for 1. Chi-Square:
Average accuracy: 0.9561 ± 0.0

IndexError: index 1 is out of bounds for axis 0 with size 1

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from collections import defaultdict
from sklearn.feature_selection import (
    VarianceThreshold, chi2, f_classif, mutual_info_classif, 
    SelectKBest, RFE, SelectFromModel, SequentialFeatureSelector
)
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from boruta import BorutaPy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import (
    Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
)
from keras.callbacks import EarlyStopping

warnings.filterwarnings('ignore')

# Load dataset
def load_data(file_path):
    print("Loading dataset...")
    df = pd.read_csv(file_path)
    X = df.drop(columns=['COVID-19'])
    y = df['COVID-19']
    print(f"Dataset loaded: {X.shape[0]} samples, {X.shape[1]} features")
    print(f"Class distribution: {dict(y.value_counts())}")
    return X, y

# Define all feature selection techniques
def get_feature_selectors(X, y, n_features=10):
    print("Initializing feature selection techniques...")
    feature_selectors = {
        "7. Correlation-based": None,  # Custom implementation
        "8. Sequential Forward Selection": SequentialFeatureSelector(
            RandomForestClassifier(n_estimators=50, random_state=42),
            n_features_to_select=n_features,
            direction='forward'
        ),
        "9. XGBoost Importance": SelectFromModel(
            XGBClassifier(n_estimators=100, random_state=42), max_features=n_features
        ),
        "10. LightGBM Importance": SelectFromModel(
            LGBMClassifier(n_estimators=100, random_state=42), max_features=n_features
        )
    }
    
    # We've already completed these techniques, so let's skip them
    completed_techniques = [
        "1. Chi-Square",
        "2. Mutual Information",
        "3. Recursive Feature Elimination",
        "4. Lasso",
        "5. Random Forest Importance",
        "6. Boruta"
    ]
    
    return feature_selectors, completed_techniques

# Function to select top features using each technique
def select_features(X, y, technique_name, selector, n_features=10):
    print(f"Selecting features using {technique_name}...")
    feature_names = X.columns.tolist()
    
    # Handle special case for Correlation-based selection
    if technique_name == "7. Correlation-based":
        correlations = []
        for col in X.columns:
            try:
                # Fixed code to handle features with only one unique value
                cross_tab = pd.crosstab(X[col], y, normalize='columns')
                # Check if there are enough rows in the crosstab
                if cross_tab.shape[0] > 1 and 1 in cross_tab.index and 0 in cross_tab.index:
                    corr = np.abs(cross_tab.loc[1, 1] - cross_tab.loc[1, 0])
                else:
                    # If not enough rows, use a small correlation value
                    corr = 0.0001
            except Exception as e:
                print(f"Error processing feature {col}: {str(e)}")
                corr = 0.0001
                
            correlations.append((col, corr))
        
        correlations.sort(key=lambda x: x[1], reverse=True)
        selected_features = [item[0] for item in correlations[:n_features]]
        feature_importances = [item[1] for item in correlations[:n_features]]
        
    elif technique_name == "6. Boruta":
        X_array = X.values
        selector.fit(X_array, y)
        selected_mask = selector.support_
        ranking = selector.ranking_
        
        feature_ranking = [(feature, rank) for feature, rank, mask in 
                          zip(feature_names, ranking, selected_mask) if mask]
        feature_ranking.sort(key=lambda x: x[1])
        
        if len(feature_ranking) < n_features:
            additional = [(f, r) for f, r, m in 
                         zip(feature_names, ranking, selected_mask) if not m]
            additional.sort(key=lambda x: x[1])
            feature_ranking.extend(additional[:n_features-len(feature_ranking)])
        
        feature_ranking = feature_ranking[:n_features]
        selected_features = [item[0] for item in feature_ranking]
        feature_importances = [1.0/item[1] for item in feature_ranking]
    
    else:
        try:
            selector.fit(X, y)
            selected_mask = selector.get_support()
            selected_features = [f for f, selected in zip(feature_names, selected_mask) if selected]
            
            if hasattr(selector, 'estimator_') and hasattr(selector.estimator_, 'feature_importances_'):
                feature_importances = selector.estimator_.feature_importances_[selected_mask]
            elif hasattr(selector, 'scores_'):
                feature_importances = selector.scores_[selected_mask]
            else:
                feature_importances = np.ones(len(selected_features))
                
        except Exception as e:
            print(f"Error with {technique_name}: {str(e)}")
            selected_features = feature_names[:n_features]
            feature_importances = np.ones(n_features)
    
    if len(selected_features) > n_features:
        selected_features = selected_features[:n_features]
        feature_importances = feature_importances[:n_features]
    elif len(selected_features) < n_features:
        remaining = [f for f in feature_names if f not in selected_features]
        selected_features.extend(remaining[:n_features-len(selected_features)])
        feature_importances = list(feature_importances) + [0] * (n_features - len(feature_importances))
    
    print(f"Top {len(selected_features)} features selected by {technique_name}:")
    for i, (feature, importance) in enumerate(zip(selected_features, feature_importances)):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    return selected_features, feature_importances

# Build the CNN model for a specific set of features
def build_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2, padding='same'),
        Dropout(0.2),
        
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2, padding='same'),
        Dropout(0.3),
        
        Flatten(),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Train and evaluate model with k-fold cross validation
def train_and_evaluate(X, y, selected_features, technique_name, k=5):
    print(f"\nTraining CNN with features selected by {technique_name}")
    
    X_selected = X[selected_features].values
    X_selected = X_selected.reshape(X_selected.shape[0], X_selected.shape[1], 1)
    y_values = y.values
    
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'auc': []
    }
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=0,
        mode='min',
        restore_best_weights=True
    )
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X_selected)):
        print(f"Training fold {fold+1}/{k}...")
        
        X_train, X_test = X_selected[train_idx], X_selected[test_idx]
        y_train, y_test = y_values[train_idx], y_values[test_idx]
        
        model = build_cnn_model((X_selected.shape[1], 1))
        model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            verbose=0,
            validation_split=0.2,
            callbacks=[early_stop]
        )
        
        y_pred = model.predict(X_test)
        y_pred_classes = (y_pred > 0.5).astype(int)
        
        metrics['accuracy'].append(accuracy_score(y_test, y_pred_classes))
        metrics['precision'].append(precision_score(y_test, y_pred_classes))
        metrics['recall'].append(recall_score(y_test, y_pred_classes))
        metrics['f1'].append(f1_score(y_test, y_pred_classes))
        try:
            metrics['auc'].append(roc_auc_score(y_test, y_pred))
        except:
            metrics['auc'].append(0.5)
    
    avg_metrics = {metric: np.mean(values) for metric, values in metrics.items()}
    std_metrics = {metric: np.std(values) for metric, values in metrics.items()}
    
    print(f"\nResults for {technique_name}:")
    for metric, value in avg_metrics.items():
        print(f"Average {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")
    
    return avg_metrics

# Function to add previously completed results
def get_completed_results():
    # Create dictionary of results from the completed techniques
    completed_results = {
        "1. Chi-Square": {
            'accuracy': 0.9561, 
            'precision': 0.9794, 
            'recall': 0.9319,
            'f1': 0.9550,
            'auc': 0.9955
        },
        "2. Mutual Information": {
            'accuracy': 0.9612,
            'precision': 0.9805,
            'recall': 0.9411,
            'f1': 0.9603,
            'auc': 0.9958
        },
        "3. Recursive Feature Elimination": {
            'accuracy': 0.9552,
            'precision': 0.9767,
            'recall': 0.9327,
            'f1': 0.9541,
            'auc': 0.9938
        },
        "4. Lasso": {
            'accuracy': 0.9464,
            'precision': 0.9675,
            'recall': 0.9241,
            'f1': 0.9449,
            'auc': 0.9924
        },
        "5. Random Forest Importance": {
            'accuracy': 0.9525,
            'precision': 0.9686,
            'recall': 0.9352,
            'f1': 0.9516,
            'auc': 0.9926
        },
        "6. Boruta": {
            'accuracy': 0.9635,
            'precision': 0.9770,
            'recall': 0.9495,
            'f1': 0.9630,
            'auc': 0.9950
        }
    }
    
    # Create dictionary of top features from completed techniques
    completed_features = {
        "1. Chi-Square": [
            "Breathing Problem", "Sore throat", "Running Nose", "Headache", 
            "Heart Disease", "Diabetes", "Hyper Tension", "Gastrointestinal",
            "Attended Large Gathering", "Family working in Public Exposed Places"
        ],
        "2. Mutual Information": [
            "Breathing Problem", "Sore throat", "Running Nose", "Headache", 
            "Heart Disease", "Diabetes", "Hyper Tension", "Gastrointestinal",
            "Attended Large Gathering", "Family working in Public Exposed Places"
        ],
        "3. Recursive Feature Elimination": [
            "Breathing Problem", "Fever", "Dry Cough", "Sore throat", 
            "Running Nose", "Headache", "Hyper Tension", "Fatigue",
            "Attended Large Gathering", "Family working in Public Exposed Places"
        ],
        "4. Lasso": [
            "Breathing Problem", "Sore throat", "Hyper Tension",
            "Attended Large Gathering", "Family working in Public Exposed Places",
            "Fever", "Dry Cough", "Running Nose", "Asthma", "Chronic Lung Disease"
        ],
        "5. Random Forest Importance": [
            "Breathing Problem", "Sore throat", "Attended Large Gathering", 
            "Family working in Public Exposed Places", "Fever", "Dry Cough",
            "Running Nose", "Asthma", "Chronic Lung Disease", "Headache"
        ],
        "6. Boruta": [
            "Breathing Problem", "Sore throat", "Running Nose", "Chronic Lung Disease",
            "Headache", "Heart Disease", "Diabetes", "Hyper Tension", "Gastrointestinal",
            "Attended Large Gathering"
        ]
    }
    
    return completed_results, completed_features

# Plot comparison bar chart
def plot_comparison(all_results):
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    
    sorted_techniques = sorted(
        all_results.keys(),
        key=lambda x: all_results[x]['accuracy'],
        reverse=True
    )
    
    plt.figure(figsize=(15, 10))
    bar_width = 0.15
    index = np.arange(len(sorted_techniques))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    for i, metric in enumerate(metrics_to_plot):
        values = [all_results[technique][metric] for technique in sorted_techniques]
        plt.bar(
            index + i * bar_width, 
            values, 
            bar_width, 
            label=metric.capitalize(),
            color=colors[i]
        )
    
    plt.xlabel('Feature Selection Technique', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.title('Comparison of Feature Selection Techniques', fontsize=14)
    plt.xticks(
        index + bar_width * 2, 
        [t.split('. ')[1] if '. ' in t else t for t in sorted_techniques],
        rotation=45,
        ha='right'
    )
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)
    plt.ylim(0, 1.0)
    plt.tight_layout()
    
    plt.savefig('feature_selection_comparison.png', dpi=300, bbox_inches='tight')
    print("Comparison chart saved as 'feature_selection_comparison.png'")
    plt.close()

# Main function to run the whole process
def main():
    file_path = "OneDrive/Desktop/Research_Capstone_Project/preprocessed_covid500_final.csv"  # Update with your path
    X, y = load_data(file_path)
    
    feature_selectors, completed_techniques = get_feature_selectors(X, y)
    completed_results, completed_features = get_completed_results()
    
    all_results = completed_results.copy()
    all_selected_features = completed_features.copy()
    
    for technique_name, selector in feature_selectors.items():
        if technique_name in completed_techniques:
            print(f"\nSkipping {technique_name} - already completed")
            continue
            
        print("\n" + "="*50)
        print(f"Processing {technique_name}")
        print("="*50)
        
        selected_features, _ = select_features(X, y, technique_name, selector)
        all_selected_features[technique_name] = selected_features
        
        results = train_and_evaluate(X, y, selected_features, technique_name)
        all_results[technique_name] = results
    
    plot_comparison(all_results)
    
    print("\n" + "="*50)
    print("FINAL SUMMARY")
    print("="*50)
    
    sorted_techniques = sorted(
        all_results.keys(),
        key=lambda x: all_results[x]['accuracy'],
        reverse=True
    )
    
    print("\nTechniques ranked by accuracy:")
    for i, technique in enumerate(sorted_techniques):
        print(f"{i+1}. {technique}: {all_results[technique]['accuracy']:.4f}")
    
    best_technique = sorted_techniques[0]
    print(f"\nBest performing technique: {best_technique}")
    print(f"Top features selected by {best_technique}:")
    for feature in all_selected_features[best_technique]:
        print(f"- {feature}")

if __name__ == "__main__":
    main()

Loading dataset...
Dataset loaded: 37028 samples, 20 features
Class distribution: {1.0: 18514, 0.0: 18514}
Initializing feature selection techniques...

Processing 7. Correlation-based
Selecting features using 7. Correlation-based...
Top 10 features selected by 7. Correlation-based:
1. Sore throat: 0.8626
2. Breathing Problem: 0.7603
3. Attended Large Gathering: 0.6464
4. Family working in Public Exposed Places: 0.6421
5. Diabetes: 0.4481
6. Heart Disease: 0.3999
7. Gastrointestinal : 0.3950
8. Hyper Tension: 0.3846
9. Running Nose: 0.3583
10. Headache: 0.3389

Training CNN with features selected by 7. Correlation-based
Training fold 1/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Training fold 2/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Training fold 3/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Training fold 4/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Training fold 5/5...
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step

Results for 7. Correlation-based:
Average accuracy: 0.9562 ± 